## Import Library

In [4]:
%pip install tensorflow setuptools pillow scipy ipywidgets


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import os

## Setup Dataset

In [6]:
base_dir = 'Dataset/fruit/'
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'val')
test_dir = os.path.join(base_dir, 'test')

print(train_dir, val_dir, test_dir)

Dataset/fruit/train Dataset/fruit/val Dataset/fruit/test


## Data Preprocessing

In [7]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1.0/255)
test_datagen = ImageDataGenerator(rescale=1.0/255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

Found 560 images belonging to 7 classes.
Found 70 images belonging to 7 classes.


Found 70 images belonging to 7 classes.


## Load Resnet50 Model

In [8]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

## Custom Layer

In [9]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
num_classes = train_generator.num_classes
output_layer = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output_layer)

## Freeze Base Model Layer

In [10]:
for layer in base_model.layers:
    layer.trainable = False

## Compile Model

In [11]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

## Train Model

In [14]:
history = model.fit(
    train_generator,
    epochs=15,
    validation_data=val_generator
)

Epoch 1/15
18/18 ━━━━━━━━━━━━━━━━━━━━ 17s 793ms/step - accuracy: 0.9868 - loss: 0.0480 - val_accuracy: 0.9571 - val_loss: 0.3187
Epoch 2/15
18/18 ━━━━━━━━━━━━━━━━━━━━ 17s 776ms/step - accuracy: 0.9851 - loss: 0.0400 - val_accuracy: 0.9571 - val_loss: 0.2848
Epoch 3/15
18/18 ━━━━━━━━━━━━━━━━━━━━ 17s 756ms/step - accuracy: 0.9843 - loss: 0.0376 - val_accuracy: 0.9429 - val_loss: 0.4906
Epoch 4/15
18/18 ━━━━━━━━━━━━━━━━━━━━ 17s 802ms/step - accuracy: 0.9855 - loss: 0.0564 - val_accuracy: 0.9571 - val_loss: 0.3380
Epoch 5/15
18/18 ━━━━━━━━━━━━━━━━━━━━ 16s 749ms/step - accuracy: 0.9980 - loss: 0.0094 - val_accuracy: 0.9571 - val_loss: 0.3590
Epoch 6/15
18/18 ━━━━━━━━━━━━━━━━━━━━ 17s 791ms/step - accuracy: 0.9947 - loss: 0.0159 - val_accuracy: 0.9429 - val_loss: 0.4451
Epoch 7/15
18/18 ━━━━━━━━━━━━━━━━━━━━ 17s 772ms/step - accuracy: 0.9951 - loss: 0.0089 - val_accuracy: 0.9571 - val_loss: 0.4000
Epoch 8/15
18/18 ━━━━━━━━━━━━━━━━━━━━ 17s 797ms/step - accuracy: 0.9911 - loss: 0.0289 - val_accu

## Evaluate Model

In [15]:
loss, accuracy = model.evaluate(test_generator)
print(f'Test accuracy: {accuracy*100:.2f}%')

3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 329ms/step - accuracy: 0.9740 - loss: 0.0581
Test accuracy: 97.14%


In [23]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Define image size
img_height, img_width = 224, 224

# Load and preprocess the image
img_path = "test/salak.jpg"
img = image.load_img(img_path, target_size=(img_height, img_width))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.0

# Predict the class of the image
predictions = model.predict(img_array)
predicted_class = np.argmax(predictions[0])

# Assuming you have a class indices dictionary from your train_generator
class_labels = list(train_generator.class_indices.keys())

print(f'Predicted class: {class_labels[predicted_class]}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 214ms/step
Predicted class: Salacca zalacca
